In [5]:
import pyodbc
import pandas as pd
from datetime import datetime
from scipy.stats import weibull_min
from scipy.stats import power_divergence

# Threshold DateTime and Hour diff for gap determination
minGapSize_Days = 30
minGapSize_Hours = 10

AH64E_SCORED_2013toPres = pd.read_csv('AH64E_SCORED_2013toPres.csv')  #Read all the data
AH64E_2013toPres_SCORED = AH64E_SCORED_2013toPres[
                            (AH64E_SCORED_2013toPres['SCD2'] != 'X') & 
                            (AH64E_SCORED_2013toPres['SCD3'] != 'N') & 
                            (AH64E_SCORED_2013toPres['SCD5'] != 'N')
                        ]
AH64E_2013toPres_SCORED.to_csv("output/AH64E_2013toPres_SCORED.csv")
AH64E_TailNumList = AH64E_2013toPres_SCORED['EI_SN'].unique()        #grab 
AH64E_RFGList     = AH64E_2013toPres_SCORED['RFG'].unique()
AH64E_SCD1List    = AH64E_2013toPres_SCORED['SCD1'].unique()


tmp1 = pd.DataFrame(AH64E_TailNumList,columns=['EI_SN'])
tmp2 = pd.DataFrame(AH64E_RFGList,columns=['RFG'])
tmp3 = pd.DataFrame(AH64E_SCD1List,columns=['SCD1'])


tmp1.to_csv('output/AH64E_TailNumList.csv')
tmp2.to_csv('output/AH64E_RFGList.csv')
tmp3.to_csv('output/AH64E_SCD1List.csv')

In [6]:

GapFrame = AH64E_SCORED_2013toPres.copy() #work with a copy, not the orig

GapFrame = GapFrame.drop(columns = ['MAL_EFF', 'CORR_DATE_TIME', 'EI_CORR_AGE', 
                                                                'TMMH', 'TMEN', 'TIMH', 'in_phase', 'in_qc', 
                                                                'RFG', 'SCD1', 'SCD2', 'SCD3', 'SCD4', 'SCD5', 
                                                                'SCD6', 'SCD7', 'SCD8', 'SCD9', 'PRIMARY_EVENT'])
try:
   GapFrame = GapFrame.drop(columns = ['RELEVANT_BEG_AGE.1','Unnamed: 0'])  # store as pkl and remove extra RELEVANT_BEG_AGE from sql
except:
    print("Superfluous columns removed already")

# Convert to Datetime to simplify the calculation.   
GapFrame.EVENT_DATE_TIME = pd.to_datetime(GapFrame.EVENT_DATE_TIME) #Seems to work without error on this dataset.  Will see if it continues

# Add a date difference column and give the difference between this date and the last
GapFrame['datediff']  = GapFrame.groupby('EI_SN')['EVENT_DATE_TIME'].diff() 
# Same for the Flight Hours
GapFrame['hoursdiff'] = GapFrame.groupby('EI_SN')['RELEVANT_BEG_AGE'].diff()

# Add previous hours and date to this row so that we can visually compare.  Not strictly necessary
GapFrame['previous_date'] = GapFrame.groupby('EI_SN')['EVENT_DATE_TIME'].shift(1)
GapFrame['previous_hours'] = GapFrame.groupby('EI_SN')['RELEVANT_BEG_AGE'].shift(1)

# Filter the GapFrame by min Gap Size *and* min Gap hours.   Save the result to a pkl (saves more of the structure of the Dataframe with column types) 
# 
TN_GAP_FRAME = GapFrame[
               ((GapFrame.datediff >= pd.Timedelta(str(minGapSize_Days) +  ' days')) & 
                (abs(GapFrame.hoursdiff) >= minGapSize_Hours)) ]
TN_GAP_FRAME.head()
TN_GAP_FRAME.to_pickle('output/TailNumbers_and_GapTimes.pkl')

# TN_GAP_FRAME represents each tail number and the dates between which there is no data (according to the minimums)
# The XXXX_ScoredDt_GAPS frames represent the periods of time for which there *is* data.   To convert I need to invert the time deltas
# This may not be necesssary and is a slight time sink.

AH64E_ScoredDtTm_GAPS = pd.DataFrame()
groups = TN_GAP_FRAME.groupby('EI_SN').groups.keys() #Tail Numbers are the groups
for group in groups :
    # Filter the original dataset to just the tailnumbers we care about and then add a row.
    # Gaps = 
    # [previous_date,EVENT_DATE_TIME], [previous_date,EVENT_DATE_TIME],[previous_date,EVENT_DATE_TIME].....
    # So Non-Gaps = 
    # [Min(EVENT_DATE_TIME),previous_date],[EVENT_DATE_TIME,previous_date],[EVENT_DATE_TIME,previous_date],[EVENT_DATE_TIME,max(EVENT_DATE_TIME)].....
    #
    tmpdf = TN_GAP_FRAME[TN_GAP_FRAME['EI_SN']==group]  
    extrarowdf = tmpdf.iloc[-1:]
    tmpdf = pd.concat([tmpdf,extrarowdf], ignore_index=True)
    tmpdf['StartDtTm'] = tmpdf['EVENT_DATE_TIME'].shift(1) 
    tmpdf['EndDtTm']   = tmpdf['previous_date']
    tmpdf['TailNumber'] = tmpdf['EI_SN']
    
    tmpdf.loc[0,'StartDtTm'] = GapFrame[GapFrame['EI_SN'] == group]['EVENT_DATE_TIME'].min() #             Min(EVENT_DATE_TIME)
    tmpdf.loc[tmpdf.index[-1],'EndDtTm'] = GapFrame[GapFrame['EI_SN'] == group]['EVENT_DATE_TIME'].max() # Max(EVENT_DATE_TIME)

    AH64E_ScoredDtTm_GAPS = pd.concat([AH64E_ScoredDtTm_GAPS,tmpdf[['TailNumber','StartDtTm','EndDtTm']]],ignore_index=True)  #add each TN Non-Gaps to dataframe.


AH64E_ScoredDtTm_GAPS.reset_index(inplace=True)
AH64E_ScoredDtTm_GAPS.drop('index',axis=1,inplace=True)
AH64E_ScoredDtTm_GAPS.to_csv('output/AH64E_ScoredDtTm_GAPS.csv')  # verified it contains the same data as the original.

In [42]:
WeibullFrame = AH64E_2013toPres_SCORED.copy()
try:
   WeibullFrame = WeibullFrame.drop(columns = ['RELEVANT_BEG_AGE.1','Unnamed: 0'])  # store as pkl and remove extra RELEVANT_BEG_AGE from sql
except:
    print("Superfluous columns removed already")
# Set all EventClasses according to SCD Rules
# Filter and save dataframe for each EventClass
WeibullFrame.loc[((WeibullFrame.SCD2 != 'N') & (WeibullFrame.SCD2 != 'P') & (WeibullFrame.SCD2 != 'X') & (WeibullFrame.SCD2 != 'Z') & (WeibullFrame.SCD2 != '')) & (WeibullFrame.SCD3 == 'C') & (WeibullFrame.SCD8 != 'N') & (WeibullFrame.RFG != '36B')
               ,'EventClass'] = 'EMA'
ema = WeibullFrame[WeibullFrame['EventClass'] == 'EMA']

WeibullFrame.loc[(WeibullFrame.SCD3 == 'C') & (WeibullFrame.SCD4 != 'O') & (WeibullFrame.SCD4 != 'H') & ((WeibullFrame.SCD2 == 'J') | (WeibullFrame.SCD2 == 'K') | (WeibullFrame.SCD2 == 'C') | (WeibullFrame.SCD2 == 'S') | (WeibullFrame.SCD2 == 'W') | (WeibullFrame.SCD2 == 'Q') | (WeibullFrame.SCD2 == 'U')) & ((WeibullFrame.SCD5 == '1') | (WeibullFrame.SCD5 == '2') | (WeibullFrame.SCD5 == '4')) & (WeibullFrame.RFG != '36B')
              ,'EventClass'] = 'MA'
ma = WeibullFrame[WeibullFrame['EventClass'] == 'MA']

WeibullFrame.loc[((WeibullFrame.SCD2 != 'D') & (WeibullFrame.SCD2 != 'N') & (WeibullFrame.SCD2 != 'P') & (WeibullFrame.SCD2 != 'X') & (WeibullFrame.SCD2 != 'Z') & (WeibullFrame.SCD2 != '')) & (WeibullFrame.SCD3 == 'C') & (WeibullFrame.SCD8 != 'N') & (WeibullFrame.SCD9 != 'N') & (WeibullFrame.RFG != '36B')
              ,'EventClass'] = 'MAF'
maf = WeibullFrame[WeibullFrame['EventClass'] == 'MAF']

WeibullFrame.loc[(WeibullFrame.SCD2 != 'X') & (WeibullFrame.SCD3 == 'C') & (WeibullFrame.SCD5 == 'S') & (WeibullFrame.RFG != '36B')
            ,'EventClass'] = 'SchedMaint'
schedmaint = WeibullFrame[WeibullFrame['EventClass'] == 'SchedMaint']

WeibullFrame.loc[((WeibullFrame.SCD2 != 'X') & (WeibullFrame.SCD2 != 'Z')) & (WeibullFrame.SCD3 == 'C') & ((WeibullFrame.SCD5 != 'M') & (WeibullFrame.SCD5 != 'R') & (WeibullFrame.SCD5 != 'S')) & (WeibullFrame.RFG != '36B')
            ,'EventClass'] = 'UMA'
uma = WeibullFrame[WeibullFrame['EventClass'] == 'UMA']

WeibullFrame.loc[(WeibullFrame.SCD5 != 'S') & (WeibullFrame.SCD2 != 'X') & (WeibullFrame.SCD3 == 'C')
            ,'EventClass'] = 'UnschedMaint'
unschedmaint = WeibullFrame[WeibullFrame['EventClass'] == 'UnschedMaint']

# Concatinate all the EventClasses into a big frame and remove the old frame
WeibullFrame = pd.concat([ema,ma,maf,schedmaint,uma,unschedmaint],ignore_index=True)


WeibullFrame['Key13 / RFG / EventClass'] = WeibullFrame.apply(lambda y: str(y.KEY13[:15] + y.RFG + '-' + y.EventClass + y.KEY13[14:]),axis=1)
WeibullFrame = WeibullFrame.drop(['MAL_EFF', 'CORR_DATE_TIME', 'EI_CORR_AGE', 'in_phase', 'in_qc', 'SCD1','SCD2', 'SCD3', 'SCD4', 'SCD5', 'SCD6', 'SCD7', 'SCD8', 'SCD9', 'PRIMARY_EVENT'], axis=1)
WeibullFrame = WeibullFrame.sort_values(['Key13 / RFG / EventClass'])
WeibullFrame = WeibullFrame.reset_index(drop=True)

WeibullFrame['lengthRFG'] = WeibullFrame.apply(lambda y :len(y['RFG']), axis=1)
WeibullFrame = WeibullFrame.rename(columns={'EI_SN':'TailNumber'})

WeibullFrame = WeibullFrame[['Key13 / RFG / EventClass','KEY13','TailNumber','EVENT_DATE_TIME','RELEVANT_BEG_AGE','TMMH','TMEN','TIMH','RFG','EventClass','lengthRFG']]
WeibullFrame.to_csv('output/AH64E_NewId_2013toPres_SCORED.csv')






In [37]:
TN_RFG_SCD_cnt = WeibullFrame.groupby(['TailNumber', 'RFG', 'lengthRFG', 'EventClass']).size().reset_index().rename(columns={0:'DataPoints'})
TN_RFG_SCD_cnt = TN_RFG_SCD_cnt.sort_values(['lengthRFG'], ascending = False)
TN_RFG_SCD_cnt = TN_RFG_SCD_cnt.reset_index(drop=True)
TN_RFG_SCD_cnt.to_csv('output/AH64E_TN_RFG_SCD_cnt.csv')

AH64E_NewId_2013toPres_SCORED = WeibullFrame.copy()



In [29]:
WeibullFrame

,Key13 / RFG / EventClass,KEY13,TailNumber,EVENT_DATE_TIME,RELEVANT_BEG_AGE,TMMH,TMEN,TIMH,RFG,EventClass,lengthRFG
0,AH-64E-1009002-00-SchedMaint-20170102-A-00001,AH-64E-1009002-20170102-A-00001,1009002,2017-01-02 00:00:00,5631.200195,0.8,1.0,0.0,00,SchedMaint,2
1,AH-64E-1009002-00-SchedMaint-20170102-A-00003,AH-64E-1009002-20170102-A-00003,1009002,2017-01-02 00:00:00,5631.200195,0.1,1.0,0.0,00,SchedMaint,2
2,AH-64E-1009002-00-SchedMaint-20170102-A-00004,AH-64E-1009002-20170102-A-00004,1009002,2017-01-02 00:00:00,5631.200195,0.1,1.0,0.0,00,SchedMaint,2
3,AH-64E-1009002-00-SchedMaint-20170103-A-00001,AH-64E-1009002-20170103-A-00001,1009002,2017-01-03 00:00:00,5631.200195,0.2,1.0,0.0,00,SchedMaint,2
4,AH-64E-1009002-00-SchedMaint-20170103-A-00002,AH-64E-1009002-20170103-A-00002,1009002,2017-01-03 00:00:00,5631.200195,0.2,1.0,0.0,00,SchedMaint,2
...,...,...,...,...,...,...,...,...,...,...,...
424446,AH-64E-1903256-39F01-MAF-20200116-A-00170,AH-64E-1903256-20200116-A-00170,1903256,2020-01-16 00:00:00,25.100000,0.5,2.0,0.1,39F01,MAF,5
424447,AH-64E-1903256-39F01-UMA-20200116-A-00170,AH-64E-1903256-20200116-A-00170,1903256,2020-01-16 00:00:00,25.100000,0.5,2.0,0.1,39F01,UMA,5
424448,AH-64E-1903256-39F01-UnschedMaint-20200116-A-0...,AH-64E-1903256-20200116-A-00170,1903256,2020-01-16 00:00:00,25.100000,0.5,2.0,0.1,39F01,UnschedMaint,5
424449,AH-64E-1903256-76C-SchedMaint-20200203-E-00100,AH-64E-1903256-20200203-E-00100,1903256,2020-02-03 00:00:00,45.099998,1.0,1.0,0.0,76C,SchedMaint,3


In [28]:
temp = WeibullFrame.groupby(['TailNumber', 'RFG', 'lengthRFG', 'EventClass']).size().reset_index().rename(columns={0:'DataPoints'})
temp = temp.sort_values(['lengthRFG'], ascending = False)
temp = temp.reset_index(drop=True)
temp.DataPoints.values 

array([   2,    3,    3, ...,  288,    1, 1862])

In [17]:
NewIdAH64E_2013toPres_SCORED = WeibullFrame.copy()
# Work through the data that doesnt have at least 2 datapoints and trim RFGs
LoopCount = 1 # Initialize the loop counter
LoopCount1 = 0 # Initialize the loop counter part 2
DoneTrimming1 = 0 # Initialize this flag for when we have finished trimming the TN/RFG/EvCl combinations that have only one datapoint
PrintLess = list(range(0, 10000000, 100)) # use this list so we don't print status on every loop
lstTrimmedInfo = [] # Initialize the list
while len(TN_RFG_SCD_cnt) > 0 and TN_RFG_SCD_cnt.iloc[0,2] > 2: # looping as long as there's data in TN_RFG_SCD_cnt, and the first row's RFG is longer than 2 characters
    prntLENGTH = len(TN_RFG_SCD_cnt) # just saving this to a variable for status print in next couple of lines
    print(LoopCount, '--', prntLENGTH, '--', datetime.now()) # printing this string as the status
    
    if LoopCount != 1: # skip this section of the loop on the first iteration
        # Look at first RFG in sparse data list to try to trim it in the Data
        WhileLoopIdx = 0 # Index the while loop
        LongestLength = TN_RFG_SCD_cnt.iloc[WhileLoopIdx,2] # Save the longest RFG length to a variable, cause we want to trim all of the RFGs that are of this same length
        CurrentLength = TN_RFG_SCD_cnt.iloc[WhileLoopIdx,2] # This is the length of the line we are on in the data, so we can check that we are still trimming the longest RFGs on this loop
        while CurrentLength == LongestLength and TN_RFG_SCD_cnt.iloc[WhileLoopIdx,2] > 2: # as long as we are still looking at the longest length of RFGs and the first row's RFG length is greater than 2 (meaning it has space for trimming)
            lstTrimIndex = [] # Initialize/Reset the list
            curTailNumber = TN_RFG_SCD_cnt.iloc[WhileLoopIdx,0] # save the first row's Tail Number to a variable
            curRFG = TN_RFG_SCD_cnt.iloc[WhileLoopIdx,1] # save the first row's RFG to a variable
            curEventClass = TN_RFG_SCD_cnt.iloc[WhileLoopIdx,3] # save the first row's Event Classification to a variable
            lstTrimmedInfo.append([curTailNumber, curRFG, curEventClass]) # appending to a list to capture RFGs being trimmed
            RFG_DataPoints = TN_RFG_SCD_cnt.iloc[WhileLoopIdx,4] # save the first row's number of datapoints to a variable
            lstTrimIndex = NewIdAH64E_2013toPres_SCORED[(NewIdAH64E_2013toPres_SCORED.TailNumber == curTailNumber) & (NewIdAH64E_2013toPres_SCORED.RFG == curRFG) & (NewIdAH64E_2013toPres_SCORED.EventClass == curEventClass)].index.tolist() # look through data and grab indexes of lines that match what we're looking for
            if len(lstTrimIndex) != RFG_DataPoints: # This is just to make sure there wasn't some mismatch between the number of indexes that were found and the number of datapoints we were supposed to find
                AAAAAA = ('***  ', curRFG, '-', curEventClass, '--', len(lstTrimIndex), ' != ', RFG_DataPoints) # so we just print an error message
                break # and we break the loop
            # loop to decide how many characters to trim off the RFG
            for a in range(1, 15): # 15 is just an arbitrary number that is bigger than we need
                if AH64E_SystemRFGs.rfg.isin([curRFG[:(len(curRFG) - a)]]).any(): # if we trim off 'a' chars, is the trimmed RFG in the RFG list
                    break # done. break the loop
            # Find where this TN/RFG/EventClass combo is in the data and trim the RFG
            for r in range(len(lstTrimIndex)): # now we're gonna loop through the actual data
                curIndex = lstTrimIndex[r] # just saving the current index of where we are in the loop through the list of indeces
                TrimChar = len(curRFG) - a # we want to pull 'a' characters off of the end of the RFG ('a' is determined in the preceding for loop)
                NewIdAH64E_2013toPres_SCORED.iloc[curIndex,9] = NewIdAH64E_2013toPres_SCORED.iloc[curIndex,9][:TrimChar] # perform the trimming of the RFG
                NewIdAH64E_2013toPres_SCORED.iloc[curIndex,11] = NewIdAH64E_2013toPres_SCORED.iloc[curIndex,11] - a # update the Length of RFG column to match the newly trimmed RFG
            WhileLoopIdx = WhileLoopIdx + 1 # Add one to the index for the next loop
            CurrentLength = TN_RFG_SCD_cnt.iloc[WhileLoopIdx,2] # Update this value for the next loop
    elif LoopCount == 1:
        LoopCount = LoopCount + LoopCount1 # do this to account for resetting the loopcount after finishing trimming combos with only 1 datapoint
    
    # Get the new Counts on the combinations of RFGs and EventClass
    TN_RFG_SCD_cnt = NewIdAH64E_2013toPres_SCORED.groupby(['TailNumber', 'RFG', 'lengthRFG', 'EventClass']).size().reset_index().rename(columns={0:'DataPoints'}) # count the number of occurences of each combination in the data
    TN_RFG_SCD_cnt = TN_RFG_SCD_cnt.sort_values(['lengthRFG'], ascending = False) # sort by the length of the RFG, from largest to smallest
    TN_RFG_SCD_cnt = TN_RFG_SCD_cnt.reset_index(drop=True) # reset the indexes to be aligned with new sorting
    
# |||||| ###################################################################################################################################### |||||| #
# |||||| ######################                  this is where the gap issue analysis is being implemented                  ################### |||||| #
# vvvvvv ###################################################################################################################################### vvvvvv #
# this whole section has to be done in the while loop because we are updating the actual data when RFGs are trimmed, this affects how the gap issues are found
    
    # Get lines from TN_RFG_SCD_cnt and NewIdAH64E_2013toPres_SCORED that are TailNumbers listed in AH64E_ScoredDtTm_GAPS
    GapTails = AH64E_ScoredDtTm_GAPS.TailNumber.tolist() # make a list of tail numbers that have gaps
    lstGAPS_Data = NewIdAH64E_2013toPres_SCORED[(NewIdAH64E_2013toPres_SCORED.TailNumber.isin(GapTails))].index.tolist() # make a list of the indexes from the data that map to tail numbers with gaps
    lstGAPS_Params = TN_RFG_SCD_cnt[(TN_RFG_SCD_cnt.TailNumber.isin(GapTails))].index.tolist() # make a list of the indexes from the counts that map to tail numbers with gaps


    Gaps_WeibullData = NewIdAH64E_2013toPres_SCORED.iloc[lstGAPS_Data] # save all the data where there is a gap
    Gaps_TN_RFG_SCD_cnt = TN_RFG_SCD_cnt.iloc[lstGAPS_Params] # save all the counts where there is a gap
    Gaps_WeibullData = Gaps_WeibullData.sort_values(['TailNumber', 'RFG', 'EventClass', 'KEY13']) # resort the data
    Gaps_TN_RFG_SCD_cnt = Gaps_TN_RFG_SCD_cnt.sort_values(['DataPoints']) # resort the counts by the number of datapoints
    Gaps_WeibullData = Gaps_WeibullData.reset_index(drop=True) # reset the indexes after new sorting
    Gaps_TN_RFG_SCD_cnt = Gaps_TN_RFG_SCD_cnt.reset_index(drop=True) # reset the indexes after new sorting
    
    # Add 2 columns to Gaps_TN_RFG_SCD_cnt to record Date-Range for data from NewIdAH64E_2013toPres_SCORED
    lstGapStartDtTm = [] # initialize the list
    lstGapEndDtTm = [] # initialize the list
    for c in range(len(Gaps_TN_RFG_SCD_cnt)): # gonna loop through the counts for the gaps
        curTailNumber = Gaps_TN_RFG_SCD_cnt.iloc[c,0] # save the current tail number to a variable
        curRFG = Gaps_TN_RFG_SCD_cnt.iloc[c,1] # save the current RFG to a variable
        curEventClass = Gaps_TN_RFG_SCD_cnt.iloc[c,3] # save the current event classification to a variable
        lstCurrentIndexes = Gaps_WeibullData[(Gaps_WeibullData.TailNumber == curTailNumber) & (Gaps_WeibullData.RFG == curRFG) & (Gaps_WeibullData.EventClass == curEventClass)].index.tolist() 
        # look through data and grab indexes of lines that match what we're looking for
        lstGapStartDtTm.append(Gaps_WeibullData.iloc[lstCurrentIndexes[0],3]) # grab the date-time from the data for the first index of the lines that matched the combination of current variables and save it to the START list
        lstGapEndDtTm.append(Gaps_WeibullData.iloc[lstCurrentIndexes[len(lstCurrentIndexes)-1],3]) # grab the date-time from the data for the last index of the lines that matched the combination of current variables and save it to the END list
    
    Gaps_TN_RFG_SCD_cnt['DataStart'] = lstGapStartDtTm # add the START list as a new column
    Gaps_TN_RFG_SCD_cnt['DataEnd'] = lstGapEndDtTm # add the END list as a new column
    
    # Check to see if there is common time between the Gap's date-range and the Data's date-range
    lstGapIssue = [] # initialize the list
    for d in range(len(Gaps_TN_RFG_SCD_cnt)): # gonna loop through the counts for the gaps again
        curTailNumber = Gaps_TN_RFG_SCD_cnt.iloc[d,0] # save the current tail number to a variable
        curTNindex = AH64E_ScoredDtTm_GAPS[(AH64E_ScoredDtTm_GAPS.TailNumber == curTailNumber)].index.tolist() # look through AH64E_ScoredDtTm_GAPS and grab indexes of lines that match what we're looking for
        if len(curTNindex) == 1: # check to see that we actually found the current TN in the list of TNs that have gaps
            # When there's a data gap, there will not be any datapoints within the gap range
            # The only scenario where the gap is an issue is when the Beginning Date-Time is before the Gap and the Ending Date-Time is after the Gap
            if AH64E_ScoredDtTm_GAPS.iloc[curTNindex[0],1] >= Gaps_TN_RFG_SCD_cnt.iloc[d,5] and AH64E_ScoredDtTm_GAPS.iloc[curTNindex[0],2] <= Gaps_TN_RFG_SCD_cnt.iloc[d,6]: # check to see if the gap resides in the actual data's date range
                lstGapIssue.append(1) # if it does, flag it
            else:
                lstGapIssue.append(0) # if not, don't flag it
        else:
            lstGapIssue.append(0) # if not, don't flag it
    
    # Let's just look at the ones where the gap is an issue
    Gaps_TN_RFG_SCD_cnt['GapIssue'] = lstGapIssue # take the list of flags and add it as a column to the counts df
    lstDataGapIssues = Gaps_TN_RFG_SCD_cnt[(Gaps_TN_RFG_SCD_cnt.GapIssue == 1)].index.tolist() # get the indexes of the ones that have been flagged as having a gap-issue
    GapIssues_TN_RFG_SCD_cnt = Gaps_TN_RFG_SCD_cnt.iloc[lstDataGapIssues] # use the indexes to make a DF for just the ones with a gap issue
    GapIssues_TN_RFG_SCD_cnt = GapIssues_TN_RFG_SCD_cnt.sort_values(['DataPoints']) # re-sort
    GapIssues_TN_RFG_SCD_cnt = GapIssues_TN_RFG_SCD_cnt.reset_index(drop=True) # re-index
    
    # Need to use the flags on the dataframe we are using for trimming, subtract 1 from the datapoints that lose an interval because of the gap
    for e in range(len(GapIssues_TN_RFG_SCD_cnt)): # gonna loop through the ones that need will lose a datapoint because of the gap
        curTailNumber = GapIssues_TN_RFG_SCD_cnt.iloc[e,0] # save the current tail number to a variable
        curRFG = GapIssues_TN_RFG_SCD_cnt.iloc[e,1] # save the current RFG to a variable
        curEventClass = GapIssues_TN_RFG_SCD_cnt.iloc[e,3] # save the current event classification to a variable
        lstCurrentIndex = TN_RFG_SCD_cnt[(TN_RFG_SCD_cnt.TailNumber == curTailNumber) & (TN_RFG_SCD_cnt.RFG == curRFG) & (TN_RFG_SCD_cnt.EventClass == curEventClass)].index.tolist() # look through data and grab the index of the line that matches what we're looking for
        TN_RFG_SCD_cnt.iloc[lstCurrentIndex[0],4] = TN_RFG_SCD_cnt.iloc[lstCurrentIndex[0],4] - 1 # subtract 1 from the datapoint to account for gap, will need to update this to subtract more than 1 when there is more than 1 gap

# ^^^^^^ ###################################################################################################################################### ^^^^^^ #
# |||||| ######################                  this is where the gap issue analysis is being implemented                  ################### |||||| #
# |||||| ###################################################################################################################################### |||||| #

    # Delete RFGs that have enough data for Weibull ... Re-sort and re-index
    TN_RFG_SCD_cnt = TN_RFG_SCD_cnt.sort_values(['DataPoints']) # sort by the number of datapoints
    TN_RFG_SCD_cnt = TN_RFG_SCD_cnt.reset_index(drop=True) # reset the indexes to be aligned with new sorting
    # we want to be left with only what needs to be trimmed
    if DoneTrimming1 == 1: # On the first loops we want to trim only combinations that have 1 datapoint, cause these will not be able to be used if they only have 1 datapoint after trimming (because they cannot create interval data)
        idxDELETE = (TN_RFG_SCD_cnt.DataPoints.values > 5).argmax() # find the index of the first row where there are enough datapoints
    elif DoneTrimming1 == 0:
        idxDELETE = (TN_RFG_SCD_cnt.DataPoints.values > 1).argmax() # find the index of the first row where there are enough datapoints
        if idxDELETE == 0: # if this is true, it means that there are no more with only 1 datapoint
            DoneTrimming1 = 1 # so we are done with this section of trimming, flag it
            LoopCount1 = LoopCount # Save off the loop counter
            LoopCount = 0 # Reset the loop count since we are on the new section of looping, we will add back the loop count each time to keep counting up
            
    # Here we just want to save off the lines that only have one data point -- need this to add to the sparse dataframe right after the while loop completes
    idxDELETE_JustTheOnes = (TN_RFG_SCD_cnt.DataPoints.values > 1).argmax() # find the index of the first row where there are enough datapoints
    DeleteLines_JustTheOnes = len(TN_RFG_SCD_cnt) - idxDELETE_JustTheOnes # subtract this index from the length of the whole dataframe, this will give the lines to be deleted
    TN_RFG_SCD_cnt_JustTheOnes = TN_RFG_SCD_cnt[:-DeleteLines_JustTheOnes] # delete the lines that have enough data and dont need to be trimmed
    
    DeleteLines = len(TN_RFG_SCD_cnt) - idxDELETE # subtract this index from the length of the whole dataframe, this will give the lines to be deleted
    TN_RFG_SCD_cnt = TN_RFG_SCD_cnt[:-DeleteLines] # delete the lines that have enough data and dont need to be trimmed
    
    if DoneTrimming1 == 1 and LoopCount != 0: # After working on trimming the combos with 1 datapoint, we need to purge the dataset of any remaining combos with only 1 datapoint
        TN_RFG_SCD_cnt = TN_RFG_SCD_cnt.sort_values(['DataPoints'], ascending = False) # sort by the number of datapoints
        TN_RFG_SCD_cnt = TN_RFG_SCD_cnt.reset_index(drop=True) # reset the indexes to be aligned with new sorting
        idxDELETE = (TN_RFG_SCD_cnt.DataPoints.values < 2).argmax() # find the index of the first row where there is still only 1 datapoint
        DeleteLines = len(TN_RFG_SCD_cnt) - idxDELETE # subtract this index from the length of the whole dataframe, this will give the lines to be deleted
        TN_RFG_SCD_cnt = TN_RFG_SCD_cnt[:-DeleteLines] # delete the lines that have still only had one data point after first round of trimming
        
        # Now we need to decide what actually needs to be trimmed in this list
        RFG_SCD_cnt = TN_RFG_SCD_cnt.groupby(['RFG', 'lengthRFG', 'EventClass']).size().reset_index().rename(columns={0:'DataPoints'}) # count the number of occurences of each combination in the data
        RFG_SCD_cnt = RFG_SCD_cnt.sort_values(['DataPoints']) # Re-sort
        RFG_SCD_cnt = RFG_SCD_cnt.reset_index(drop=True) # reset the indexes to be aligned with new sorting
        idxDELETE = (RFG_SCD_cnt.DataPoints.values > 4).argmax() # find the index of the first row where there is enough data
        DeleteLines = len(RFG_SCD_cnt) - idxDELETE # subtract this index from the length of the whole dataframe, this will give the lines to be deleted
        RFG_SCD_cnt = RFG_SCD_cnt[:-DeleteLines] # delete the lines that have enough data and dont need to be trimmed
        
        # Now get the indexes so we can find out how many intervals there are actually for the RFG/EvCl combo
        lstNeedsMoreData = [] # initialize a list for saving the indexes of the combos that will still need RFG trimming
        for h in range(len(RFG_SCD_cnt)):
            lstTotalDatapoints = [] # Initialize the list for saving all of the datapoint values
            curRFG = RFG_SCD_cnt.iloc[h,0]
            curEventClass = RFG_SCD_cnt.iloc[h,2]
            curOccurences = RFG_SCD_cnt.iloc[h,3]
            lstCurrentCountIndexes = TN_RFG_SCD_cnt[(TN_RFG_SCD_cnt.RFG == curRFG) & (TN_RFG_SCD_cnt.EventClass == curEventClass)].index.tolist() # look through data and grab indexes of lines that match what we're looking for
            lstTotalDatapoints = TN_RFG_SCD_cnt.iloc[lstCurrentCountIndexes,4]
            if (sum(lstTotalDatapoints) - curOccurences) < 5:
                lstNeedsMoreData.extend(lstCurrentCountIndexes)
        TN_RFG_SCD_cnt = TN_RFG_SCD_cnt.iloc[lstNeedsMoreData]
    
    # Sort by length of RFG
    TN_RFG_SCD_cnt = TN_RFG_SCD_cnt.sort_values(['lengthRFG'], ascending = False) # sort by the length of the RFG, from largest to smallest -- doing this to always begin trimming from the most specific RFG (specific --> general)
    TN_RFG_SCD_cnt = TN_RFG_SCD_cnt.reset_index(drop=True) # reset the indexes to be aligned with new sorting
    
    if (DoneTrimming1 == 0 and TN_RFG_SCD_cnt.iloc[0,2] < 3) or LoopCount == 0: # We want to check if RFGs are still long enough to be trimmed
        DoneTrimming1 = 1
        if LoopCount != 0: # if this is equal to zero, then we alread saved LoopCount to LoopCount1 and reset LoopCount to 0... we don't want to make LoopCount1 also reset to 0
            LoopCount1 = LoopCount # Save off the loop counter
        LoopCount = 0 # Reset the loop count since we are on the new section of looping, we will add back the loop count each time to keep counting up
        # Need to reset the dataframe so that we stay in the while loop after finishing with the ONES
        TN_RFG_SCD_cnt = NewIdAH64E_2013toPres_SCORED.groupby(['TailNumber', 'RFG', 'lengthRFG', 'EventClass']).size().reset_index().rename(columns={0:'DataPoints'}) # count the number of occurences of each combination in the data
        TN_RFG_SCD_cnt = TN_RFG_SCD_cnt.sort_values(['lengthRFG'], ascending = False) # sort by the length of the RFG, from largest to smallest
        TN_RFG_SCD_cnt = TN_RFG_SCD_cnt.reset_index(drop=True) # reset the indexes to be aligned with new sorting
        # This will only happen once (the first time through where we have finished with the ONES) -- it will not affect next loop because LoopCount was reset and it will pass over trimming section as if it were the first loop again
    
    # Increase the LoopCount by 1 to record another iteration has occured
    LoopCount = LoopCount + 1 # plus-one, and back to the start of the while loop, unless it's time to break out of the loop
    # looping as long as there's data in TN_RFG_SCD_cnt (at this point of loop, we have deleted the lines that have enough datapoints for the analysis), and the first row's RFG is longer than 2 characters (meaning it can be trimmed further)
    # The most likely scenario is that there will still be lines that do not have enough datapoints even after being trimmed to an RFG of only 2 characters, meaning the while loop will break when the first (largest due to sorting) RFG in the dataframe is only 2 characters long


1 -- 59794 -- 2022-04-24 12:51:20.533657
2 -- 40451 -- 2022-04-24 13:21:45.731694


KeyboardInterrupt: 

In [43]:

def shorten_rfg(rfg):
    if (len(rfg) <=2 ):
        return rfg
    if (rfg[-1].isalpha()) :
        print("Shortening " + rfg + " -> " + rfg[:-1])
        return rfg[:-1]
    else :
        print("Shortening " + rfg + " -> " + rfg[:-2])
        return rfg[:-2]

WeibullFrame[WeibullFrame['RFG'].map(WeibullFrame['RFG'].value_counts()) <= 4 ]['RFG'].map(len).max()

while ((WeibullFrame[WeibullFrame['RFG'].map(WeibullFrame['RFG'].value_counts()) <= 4 ].shape[0] > 0) & (WeibullFrame[WeibullFrame['RFG'].map(WeibullFrame['RFG'].value_counts()) <= 4 ]['RFG'].map(len).max() > 2 )):
     #print ("Shortening the following RFGs:  " + str(WeibullFrame[WeibullFrame['RFG'].map(WeibullFrame['RFG'].value_counts()) == 1]['RFG']))
     WeibullFrame.loc[WeibullFrame['RFG'].map(WeibullFrame['RFG'].value_counts()) <= 4, 'RFG'] = WeibullFrame.loc[WeibullFrame['RFG'].map(WeibullFrame['RFG'].value_counts()) <= 4]['RFG'].apply(lambda y : shorten_rfg(y))







Shortening 11A02C02 -> 11A02C
Shortening 11A02C02 -> 11A02C
Shortening 11A02C02 -> 11A02C
Shortening 12C03 -> 12C
Shortening 12C03 -> 12C
Shortening 04A05 -> 04A
Shortening 04A05 -> 04A
Shortening 04A05 -> 04A
Shortening 05A01E -> 05A01
Shortening 05A01E -> 05A01
Shortening 05A01E -> 05A01
Shortening 06G01H01 -> 06G01H
Shortening 06G01H01 -> 06G01H
Shortening 06G01H01 -> 06G01H
Shortening 08A05C -> 08A05
Shortening 08A05C -> 08A05
Shortening 08A05C -> 08A05
Shortening 08A05C -> 08A05
Shortening 11C25E -> 11C25
Shortening 13A01 -> 13A
Shortening 13A01 -> 13A
Shortening 13A01 -> 13A
Shortening 15D02 -> 15D
Shortening 19E04F01 -> 19E04F
Shortening 19E04F01 -> 19E04F
Shortening 19E04F01 -> 19E04F
Shortening 76D04 -> 76D
Shortening 76D04 -> 76D
Shortening 04A10C -> 04A10
Shortening 07B12C -> 07B12
Shortening 07B12C -> 07B12
Shortening 07B12C -> 07B12
Shortening 10E -> 10
Shortening 13A16F -> 13A16
Shortening 13A16F -> 13A16
Shortening 15A -> 15
Shortening 76A06 -> 76A
Shortening 76A06 -> 76

In [51]:
len(WeibullFrame.RFG.unique())

1116

In [119]:
# Read in CSV, just cause it's easier this way
AH64E_2013toPres_SCORED = pd.read_csv('AH64E_2013toPres_SCORED.csv')
# Rename index column that is created when saved to CSV to be placeholder for new ID column
# AH64E_2013toPres_SCORED = AH64E_2013toPres_SCORED.rename(columns={"Unnamed: 0": "Key13 / RFG / EventClass"})

# Create index lists for each of the Event Classifications by the SCD1 rules in SQL
lstEMAs          = AH64E_2013toPres_SCORED[
        ((AH64E_2013toPres_SCORED.SCD2 != 'N') & 
         (AH64E_2013toPres_SCORED.SCD2 != 'P') & 
         (AH64E_2013toPres_SCORED.SCD2 != 'X') & 
         (AH64E_2013toPres_SCORED.SCD2 != 'Z') & 
         (AH64E_2013toPres_SCORED.SCD2 != '')) & 
         (AH64E_2013toPres_SCORED.SCD3 == 'C') & 
         (AH64E_2013toPres_SCORED.SCD8 != 'N') & 
         (AH64E_2013toPres_SCORED.RFG != '36B')   ].index.tolist()
lstMAs           = AH64E_2013toPres_SCORED[
       (AH64E_2013toPres_SCORED.SCD3 == 'C') & 
       (AH64E_2013toPres_SCORED.SCD4 != 'O') & 
       (AH64E_2013toPres_SCORED.SCD4 != 'H') & 
       ((AH64E_2013toPres_SCORED.SCD2 == 'J') | 
       (AH64E_2013toPres_SCORED.SCD2 == 'K') | 
       (AH64E_2013toPres_SCORED.SCD2 == 'C') | 
       (AH64E_2013toPres_SCORED.SCD2 == 'S') | 
       (AH64E_2013toPres_SCORED.SCD2 == 'W') | 
       (AH64E_2013toPres_SCORED.SCD2 == 'Q') | 
       (AH64E_2013toPres_SCORED.SCD2 == 'U')) & 
       ((AH64E_2013toPres_SCORED.SCD5 == '1') | 
       (AH64E_2013toPres_SCORED.SCD5 == '2') | 
       (AH64E_2013toPres_SCORED.SCD5 == '4')) & 
       (AH64E_2013toPres_SCORED.RFG != '36B')].index.tolist()
lstMAFs          = AH64E_2013toPres_SCORED[
    ((AH64E_2013toPres_SCORED.SCD2 != 'D') & 
    (AH64E_2013toPres_SCORED.SCD2 != 'N') & 
    (AH64E_2013toPres_SCORED.SCD2 != 'P') & 
    (AH64E_2013toPres_SCORED.SCD2 != 'X') & 
    (AH64E_2013toPres_SCORED.SCD2 != 'Z') & 
    (AH64E_2013toPres_SCORED.SCD2 != '')) & 
    (AH64E_2013toPres_SCORED.SCD3 == 'C') & 
    (AH64E_2013toPres_SCORED.SCD8 != 'N') & 
    (AH64E_2013toPres_SCORED.SCD9 != 'N') & 
    (AH64E_2013toPres_SCORED.RFG != '36B')].index.tolist()
lstSchedMaints   = AH64E_2013toPres_SCORED[
    (AH64E_2013toPres_SCORED.SCD2 != 'X') & 
    (AH64E_2013toPres_SCORED.SCD3 == 'C') & 
    (AH64E_2013toPres_SCORED.SCD5 == 'S') & 
    (AH64E_2013toPres_SCORED.RFG != '36B')].index.tolist()
lstUMAs          = AH64E_2013toPres_SCORED[
    ((AH64E_2013toPres_SCORED.SCD2 != 'X') & 
    (AH64E_2013toPres_SCORED.SCD2 != 'Z')) & 
    (AH64E_2013toPres_SCORED.SCD3 == 'C') & 
    ((AH64E_2013toPres_SCORED.SCD5 != 'M') & 
    (AH64E_2013toPres_SCORED.SCD5 != 'R') & 
    (AH64E_2013toPres_SCORED.SCD5 != 'S')) & 
    (AH64E_2013toPres_SCORED.RFG != '36B')].index.tolist()
lstUnschedMaints = AH64E_2013toPres_SCORED[
    (AH64E_2013toPres_SCORED.SCD5 != 'S') & 
    (AH64E_2013toPres_SCORED.SCD2 != 'X') & 
    (AH64E_2013toPres_SCORED.SCD3 == 'C')].index.tolist()

# Use lists to make dataframes of data specific to only the relative Event Classification
EMA_All_WeibullData = AH64E_2013toPres_SCORED.loc[AH64E_2013toPres_SCORED.index.isin(lstEMAs)]
EMA_All_WeibullData = EMA_All_WeibullData.reset_index(drop=True)
MA_All_WeibullData = AH64E_2013toPres_SCORED.loc[AH64E_2013toPres_SCORED.index.isin(lstMAs)]
MA_All_WeibullData = MA_All_WeibullData.reset_index(drop=True)
MAF_All_WeibullData = AH64E_2013toPres_SCORED.loc[AH64E_2013toPres_SCORED.index.isin(lstMAFs)]
MAF_All_WeibullData = MAF_All_WeibullData.reset_index(drop=True)
SchedMaint_All_WeibullData = AH64E_2013toPres_SCORED.loc[AH64E_2013toPres_SCORED.index.isin(lstSchedMaints)]
SchedMaint_All_WeibullData = SchedMaint_All_WeibullData.reset_index(drop=True)
UMA_All_WeibullData = AH64E_2013toPres_SCORED.loc[AH64E_2013toPres_SCORED.index.isin(lstUMAs)]
UMA_All_WeibullData = UMA_All_WeibullData.reset_index(drop=True)
UnschedMaint_All_WeibullData = AH64E_2013toPres_SCORED.loc[AH64E_2013toPres_SCORED.index.isin(lstUnschedMaints)]
UnschedMaint_All_WeibullData = UnschedMaint_All_WeibullData.reset_index(drop=True)

# Rename SCD1 column to be EventClass
EMA_All_WeibullData = EMA_All_WeibullData.rename(columns={"SCD1": "EventClass"})
MA_All_WeibullData = MA_All_WeibullData.rename(columns={"SCD1": "EventClass"})
MAF_All_WeibullData = MAF_All_WeibullData.rename(columns={"SCD1": "EventClass"})
SchedMaint_All_WeibullData = SchedMaint_All_WeibullData.rename(columns={"SCD1": "EventClass"})
UMA_All_WeibullData = UMA_All_WeibullData.rename(columns={"SCD1": "EventClass"})
UnschedMaint_All_WeibullData = UnschedMaint_All_WeibullData.rename(columns={"SCD1": "EventClass"})

# Create lists to put into EventClass columns
lstEMAstr = ['EMA'] * len(lstEMAs)
lstMAstr = ['MA'] * len(lstMAs)
lstMAFstr = ['MAF'] * len(lstMAFs)
lstSchedMaintstr = ['SchedMaint'] * len(lstSchedMaints)
lstUMAstr = ['UMA'] * len(lstUMAs)
lstUnschedMaintstr = ['UnschedMaint'] * len(UnschedMaint_All_WeibullData)

EMA_All_WeibullData['EventClass'] = lstEMAstr
MA_All_WeibullData['EventClass'] = lstMAstr
MAF_All_WeibullData['EventClass'] = lstMAFstr
SchedMaint_All_WeibullData['EventClass'] = lstSchedMaintstr
UMA_All_WeibullData['EventClass'] = lstUMAstr
UnschedMaint_All_WeibullData['EventClass'] = lstUnschedMaintstr

# Put all the data back together into one DataFrame
WeibullFrame = EMA_All_WeibullData.append(MA_All_WeibullData, ignore_index=True)
WeibullFrame = WeibullFrame.append(MAF_All_WeibullData, ignore_index=True)
WeibullFrame = WeibullFrame.append(SchedMaint_All_WeibullData, ignore_index=True)
WeibullFrame = WeibullFrame.append(UMA_All_WeibullData, ignore_index=True)
WeibullFrame = WeibullFrame.append(UnschedMaint_All_WeibullData, ignore_index=True)


print ("EMA : " + str( WeibullFrame[WeibullFrame.EventClass == 'EMA'].shape[0]))
print ("MA : " +str( WeibullFrame[WeibullFrame.EventClass == 'MA'].shape[0]))
print ("MAF : " +str( WeibullFrame[WeibullFrame.EventClass == 'MAF'].shape[0]))
print ("SchedMaint : " +str( WeibullFrame[WeibullFrame.EventClass == 'SchedMaint'].shape[0]))
print ("UMA : " +str( WeibullFrame[WeibullFrame.EventClass == 'UMA'].shape[0]))
print ("UnschedMaint : " +str( WeibullFrame[WeibullFrame.EventClass == 'UnschedMaint'].shape[0]))


EMA : 15632
MA : 2690
MAF : 3798
SchedMaint : 356442
UMA : 22942
UnschedMaint : 22947


In [121]:
WeibullFrame.KEY13.unique().shape[0]

379389